In [2]:
import os

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from model import Fitted_DT
from utils.model import get_model, get_vocoder, get_param_num
from utils.tools import to_device_eval
from dataset import Dialogue_dataset_eval
from scipy.io import wavfile
from utils.model import vocoder_infer

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:

model_name = "Fitted_v19_flowv2_length4"
preprocess_path = "./config/Fitted_DT/preprocess.yaml"
model_path = "./config/Fitted_DT/model.yaml"
train_path = "./config/Fitted_DT/train.yaml"
preprocess_config = yaml.load(open(preprocess_path, "r"), Loader=yaml.FullLoader)
model_config = yaml.load(open(model_path, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(train_path, "r"), Loader=yaml.FullLoader)

In [8]:
warmup = train_config["optimizer"]["warm_up_step"]
model = Fitted_DT(preprocess_config, model_config, warmup*2)
model1 = Fitted_DT(preprocess_config, model_config, warmup*2)
step = 1

In [9]:
torch.save(
                        {
                            "model": model.state_dict(),
                        },
                        os.path.join(
                            train_config["path"]["ckpt_path"],
                            "{}.pth.tar".format(step),
                        ),
                    )

In [12]:
ckpt_path = os.path.join(
    train_config["path"]["ckpt_path"],
    "{}.pth.tar".format(1),
)
ckpt = torch.load(ckpt_path)
model1.load_state_dict(ckpt["model"])

<All keys matched successfully>

In [17]:
def compute_model_param_diff(model1, model2):
    """
    두 모델의 파라미터 차이를 계산합니다.
    :param model1: 첫 번째 모델
    :param model2: 두 번째 모델
    :return: 두 모델의 파라미터 차이
    """
    # 두 모델의 파라미터를 가져옵니다.
    params1 = dict(model1.named_parameters())
    params2 = dict(model2.named_parameters())

    # 파라미터 차이를 저장할 딕셔너리
    param_diff = {}

    # 모델 1의 파라미터가 모델 2에 존재하는지 확인하고 차이를 계산
    for name, param1 in params1.items():
        if name in params2:
            param2 = params2[name]
            # 파라미터 차이 계산 (절대 차이 또는 상대 차이)
            diff = torch.abs(param1.data - param2.data)
            print(name, torch.sum(diff))
            param_diff[name] = diff

In [18]:
compute_model_param_diff(model, model1)

position_enc tensor(0.)
encoder.position_enc tensor(0.)
encoder.src_word_emb.weight tensor(0.)
encoder.layer_stack.0.slf_attn.w_qs.weight tensor(0.)
encoder.layer_stack.0.slf_attn.w_qs.bias tensor(0.)
encoder.layer_stack.0.slf_attn.w_ks.weight tensor(0.)
encoder.layer_stack.0.slf_attn.w_ks.bias tensor(0.)
encoder.layer_stack.0.slf_attn.w_vs.weight tensor(0.)
encoder.layer_stack.0.slf_attn.w_vs.bias tensor(0.)
encoder.layer_stack.0.slf_attn.layer_norm.weight tensor(0.)
encoder.layer_stack.0.slf_attn.layer_norm.bias tensor(0.)
encoder.layer_stack.0.slf_attn.fc.weight tensor(0.)
encoder.layer_stack.0.slf_attn.fc.bias tensor(0.)
encoder.layer_stack.0.pos_ffn.w_1.weight tensor(0.)
encoder.layer_stack.0.pos_ffn.w_1.bias tensor(0.)
encoder.layer_stack.0.pos_ffn.w_2.weight tensor(0.)
encoder.layer_stack.0.pos_ffn.w_2.bias tensor(0.)
encoder.layer_stack.0.pos_ffn.layer_norm.weight tensor(0.)
encoder.layer_stack.0.pos_ffn.layer_norm.bias tensor(0.)
encoder.layer_stack.1.slf_attn.w_qs.weight tens